<img src="dcic.png">

# Mapping Inequality in Baltimore

## Introduction 

After the 1929 Stock Market Crash, thousands of Americans were in danger of home foreclosure. To address this problem and stabilize the market, The Home Owners Loan Coperation was created under the Rooselvelt Administration to asses credibility of families in order to decide to refinance their homes.

- Redlining is the process 

- In particular Baltimore,

<img src="baltimoremap.png">

- A - 
- B - 
- C - 
- D - 

Learn more the Mapping Inequality Project [here](https://dsl.richmond.edu/panorama/redlining/#loc=4/36.71/-96.93&opacity=0.8)

- This data set
- Learning Objectives 

## 1. Aquiring Data

- something about json to csv, how we obtain our csv , rawdata

### Importing the Data and Creating a Data Frame

In [5]:
# Loads the Pandas library 
import pandas as pd

# Creates data frame (df) by reading in the Baltimore csv
df = pd.read_csv("AD_Data_BaltimoreProject.csv")
df.head()

,Form,State,City,Security_Grade,Area_Number,Terrain_Description,Favorable_Influences,Detrimental_Influences,INHABITANTS_Type,INHABITANTS_Annual_Income,...,INHABITANTS_Population_Increase,INHABITANTS_Population_Decrease,INHABITANTS_Population_Static,BUILDINGS_Types,BUILDINGS_Construction,BUILDINGS_Age,BUILDINGS_Repair,Ten_Fifteen_Desirability,Remarks,Date
0,NS FORM-8 6-1-37,Maryland,Baltimore,A,2.0,Rolling,Fairly new suburban area of homogeneous charac...,None,Substantial Middle Class,"$3000 - 5,000",...,Fast,NaN,NaN,Detached an row houses,Brick and frame,1 to 10 years,Good,Upward,A recent development with much room for expans...,"May 4,1937"
1,NS FORM-8 6-1-37,Maryland,Baltimore,A,1.0,Undulating,Very nicely planned residential area of medium...,None,"Executives, Professional Men",over $5000,...,Moderately Fast,NaN,NaN,Single family detached,Brick and Stone,12 years,Very good,Upward,Mostly fee properties. A few homes valued at $...,"May 4,1937"
2,NS FORM-8 6-1-37,Maryland,Baltimore,A,3.0,Rolling,Good residential area. Well planned.,Distance to City,"Executives, Professional Men",3500 - 7000,...,Moderately Fast,NaN,NaN,One family detached,"Brick, Stone, and Frame",1 to 20 years,Good to excellent,Upward,Principally fee property. This section lies in...,"May 4,1937"
3,NS FORM-8 6-1-37,Maryland,Baltimore,A,4.0,Level,Well planned development of fairly,None,Professional and Executives,over $5000,...,Slowly,NaN,NaN,One family,"Brick, Stone, and Frame",10 years,Good,Upward,All fee property,"May 4,1937"
4,NS FORM-8 6-1-37,Maryland,Baltimore,A,5.0,Undulating,"Desirable residential section. Good quality, m...",None,"Executives, Professional Men","$3,500 - $10,000",...,Moderately Fast,NaN,NaN,One family detached,Brick and Stone,1 to 20 years,Good,Upward,Merridale only recently developed. Prices do n...,NaN


- note on refering to the rows and columns - entities/attributes , observatons/variables

- form picture and corresponding attributes 

In [6]:
# Lists all the columns of the data frame
df.dtypes

Form                                object
State                               object
City                                object
Security_Grade                      object
Area_Number                        float64
Terrain_Description                 object
Favorable_Influences                object
Detrimental_Influences              object
INHABITANTS_Type                    object
INHABITANTS_Annual_Income           object
INHABITANTS_Foreignborn             object
INHABITANTS_F                      float64
INHABITANTS_Negro                   object
INHABITANTS_N                       object
INHABITANTS_Infiltration            object
INHABITANTS_Relief_Families         object
INHABITANTS_Population_Increase     object
INHABITANTS_Population_Decrease     object
INHABITANTS_Population_Static       object
BUILDINGS_Types                     object
BUILDINGS_Construction              object
BUILDINGS_Age                       object
BUILDINGS_Repair                    object
Ten_Fifteen

## 2. Data Cleaning 

### Handling Similar Text Values 

- Text

In [24]:
df.INHABITANTS_Foreignborn

0           No
1         None
2         None
3         None
4         None
5          NaN
6           No
7           No
8           No
9        Small
10    Very few
11          No
12          No
13          No
14          No
15     Mixture
16        None
17         Few
18          No
19          No
20        None
21        None
22          No
23          No
24          No
25         NaN
26         NaN
27         NaN
28          No
29       Small
30        None
31          No
32    Mixture 
33     Mixture
34     Mixture
35          No
36     Mixture
37     Mixture
38     Mixture
39     Mixture
40     Mixture
41     Nominal
42         NaN
43         NaN
44     Nominal
45         NaN
46         NaN
47         NaN
48         NaN
49         NaN
50    Italians
51      Polish
52     Mixture
53     Mixture
54     Mixture
55     Mixture
56     Mixture
Name: INHABITANTS_Foreignborn, dtype: object

In [31]:
# Replaces the values of 'No' with 'None'
df['INHABITANTS_Foreignborn'] = df['INHABITANTS_Foreignborn'].replace('No', 'None')

# Replaces all other values with 'Yes'
for value in df['INHABITANTS_Foreignborn']:
    if value != 'None':
        df['INHABITANTS_Foreignborn'] = df['INHABITANTS_Foreignborn'].replace(value, 'Yes')

0     None
1     None
2     None
3     None
4     None
5      Yes
6     None
7     None
8     None
9      Yes
10     Yes
11    None
12    None
13    None
14    None
15     Yes
16    None
17     Yes
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25     Yes
26     Yes
27     Yes
28    None
29     Yes
30    None
31    None
32     Yes
33     Yes
34     Yes
35    None
36     Yes
37     Yes
38     Yes
39     Yes
40     Yes
41     Yes
42     Yes
43     Yes
44     Yes
45     Yes
46     Yes
47     Yes
48     Yes
49     Yes
50     Yes
51     Yes
52     Yes
53     Yes
54     Yes
55     Yes
56     Yes
Name: INHABITANTS_Foreignborn, dtype: object

In [6]:
no_words = ['No.','no', 'none', 'No']

for word in no_words:
    df['Detrimental_Influences'] = df['Detrimental_Influences'].replace( word , 'None')

- Note on functions

In [ ]:
for word in words_to_replace:
    df['INHABITANTS_Negro'] = df['INHABITANTS_Negro'].replace( word , 'None')
    
for word in words_to_replace:
    df['INHABITANTS_Infiltration '] = df['INHABITANTS_Infiltration'].replace( word , 'None')
    

In [ ]:
small_words = ['Nominal','nominal', 'Small','small','Minimum', 'minimum','minimal', 'Very few']

for word in small_words:
    df ['INHABITANTS_Negro'] = df['INHABITANTS_Negro'].replace( word , 'Few')

for word in small_words:
    df ['INHABITANTS_Foreignborn'] = df['INHABITANTS_Foreignborn'].replace( word , 'Few')

In [17]:
def replace_values(attribute, new_word, words_to_replace):
    for word in words_to_replace:
        df[attribute] = df[attribute].replace(word, new_word)
        # print or return result ?

# example run
replace_values('INHABITANTS_Negro', 'None', no_words)

In [18]:
df.INHABITANTS_Negro

0        None
1        None
2        None
3        None
4        None
5         NaN
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18        Yes
19       None
20       None
21       None
22       None
23       None
24       None
25        NaN
26        NaN
27        NaN
28    Nominal
29       None
       ...   
31       None
32        Yes
33        Yes
34       None
35       None
36        Yes
37       None
38        Yes
39       None
40        Yes
41       None
42        NaN
43        NaN
44        NaN
45        NaN
46        NaN
47        NaN
48        NaN
49        NaN
50        Yes
51        Yes
52        Yes
53        Yes
54        Yes
55        Yes
56        NaN
57        Yes
58        NaN
59        Yes
60        NaN
Name: INHABITANTS_Negro, dtype: object

### Population 

In [ ]:
#Population Increase

### Building Age 

In [ ]:
#building age 

### Handling Dates 

In [ ]:
#Date

## 3. Data Manipulation 

- 

### Income 

In [2]:
#Annual_income
# split the range into max and min 

### Sorting 

In [ ]:
# Area Number 

### Grouping 

## 4. Data Analysis 

## 5. Data Visualization 

<img src="redl.png">